# ReAct Agent - Código Python com Anotações de Estudo


Este notebook foi desenvolvido baseado no tutorial indicado no Fastcamp de Agentes Inteligentes: https://www.youtube.com/watch?v=hKVhRA9kfeM


In [ ]:
# Instale as bibliotecas necessárias
%pip install groq python-dotenv 

In [3]:
import os
import re
from groq import Groq
from dotenv import load_dotenv

In [4]:
# Carrega as variáveis do arquivo .env
load_dotenv()

# Cria cliente da API Groq
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [5]:
# Classe Agent original
class Agent:
    def __init__(self, client, system):
        self.client = client  # Cliente da API
        self.system = system  # System prompt
        self.messages = []    # Histórico da conversa
        # Adiciona o system prompt como primeira mensagem
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})
    
    def __call__(self, message=""):
        # Permite chamar o agente como função: agent("pergunta")
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        # Faz a chamada para a API e retorna a resposta
        completion = self.client.chat.completions.create(
            messages=self.messages,
            model="llama-3.3-70b-versatile"     
        )
        return completion.choices[0].message.content

**Observação:** Para expandir minha prática, decidi trabalhar em um domínio diferente do abordado na videoaula. No meu código, usarei tools e um system prompt focado em tirar dúvidas sobre as tarefas do Fastcamp de Agentes Inteligentes.

In [6]:
# System prompt modificado para as novas tools
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_activity_title:
e.g. get_activity_title: 1
Returns the title of the activity from Fastcamp de Agentes Inteligentes based on the number

get_activity_deadline:
e.g. get_activity_deadline: Vídeo: Fundamentos de Agentes de IA (I)
Returns the deadline for the activity based on its title

Example session:

Question: What is the deadline for activity 1?
Thought: I need to first get the title of activity 1, then find its deadline
Action: get_activity_title: 1
PAUSE 

You will be called again with this:

Observation: Vídeo: Fundamentos de Agentes de IA (I)

Thought: Now I need to get the deadline for this activity
Action: get_activity_deadline: Vídeo: Fundamentos de Agentes de IA (I)
PAUSE

You will be called again with this: 

Observation: 2 dias

If you have the answer, output it as the Answer.

Answer: The deadline for activity 1 is 2 days.

Now it's your turn:
""".strip()

In [7]:
# Novas tools para o Fastcamp
def get_activity_title(activity_number: str) -> str:
    # Mapeia número da atividade para o título
    activities = {
        "1": "Vídeo: Fundamentos de Agentes de IA (I)",
        "2": "Prática: Python: Criando um ReAct Agent do Zero (I)",
        "3": "Prática: Validação de dados com Pydantic (I)",
        "4": "Vídeo: Introdução ao n8n (II)",
        "5": "Prática: Construindo um fluxo n8n(II)",
        "6": "Prática: Embedding (II)",
        "7": "Leitura: n8n (II)",
        "8": "Pratica: Agentes com Google ADK (III)",
        "9": "Leitura: Construindo Agentes Google ADK(III)",
        "10": "Pratica: Multi Agentes com ADK (III)",
        "11": "Prática: Criando agentes com ADK e Streamlit (IV)",
        "12": "Prática: Criando agentes com n8n, ADK e Whatsapp (IV)",
        "13": "Desafio: Orquestrando Agentes com Google ADK (IV)",
        "14": "Prática: Projeto Final (V)"
    }
    return activities.get(activity_number, "Atividade não encontrada")

def get_activity_deadline(activity_title: str) -> str:
    # Mapeia título da atividade para o prazo
    deadlines = {
        "Vídeo: Fundamentos de Agentes de IA (I)": "2 dias",
        "Prática: Python: Criando um ReAct Agent do Zero (I)": "4 dias",
        "Prática: Validação de dados com Pydantic (I)": "4 dias",
        "Vídeo: Introdução ao n8n (II)": "3 dias",
        "Prática: Construindo um fluxo n8n(II)": "3 dias",
        "Prática: Embedding (II)": "3 dias",
        "Leitura: n8n (II)": "3 dias",
        "Pratica: Agentes com Google ADK (III)": "4 dias",
        "Leitura: Construindo Agentes Google ADK(III)": "2 dias",
        "Pratica: Multi Agentes com ADK (III)": "4 dias",
        "Prática: Criando agentes com ADK e Streamlit (IV)": "4 dias",
        "Prática: Criando agentes com n8n, ADK e Whatsapp (IV)": "3 dias",
        "Desafio: Orquestrando Agentes com Google ADK (IV)": "5 dias",
        "Prática: Projeto Final (V)": "7 dias"
    }
    return deadlines.get(activity_title, "Prazo não encontrado")

In [8]:
# Função loop original com as novas tools
def loop(max_iterations=10, query: str = ""):
    # Cria novo agente para cada consulta
    agent = Agent(client=client, system=system_prompt)
    # Lista das ferramentas disponíveis
    tools = ["get_activity_title", "get_activity_deadline"]
    next_prompt = query
    i = 0
  
    while i < max_iterations:
        i += 1
        # Envia mensagem para o agente
        result = agent(next_prompt)
        print(result)

        # Verifica se agente quer executar uma ação
        if "PAUSE" in result and "Action" in result:
            # Extrai ação usando regex
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]  # Nome da função
            arg = action[0][1]          # Argumento

            # Executa a ferramenta se ela existir
            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"
            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        # Para quando agente dá resposta final
        if "Answer" in result:
            break


## Exemplos de Uso

In [9]:
# Testes com as novas perguntas
print("TESTANDO O REACT AGENT")
print("TESTE 1: Consultando prazo de uma atividade")
# Pergunta simples sobre uma atividade
loop(query="What is the deadline for activity 5?")

TESTANDO O REACT AGENT
TESTE 1: Consultando prazo de uma atividade
Thought: To find the deadline for activity 5, I need to first get the title of activity 5, then use this title to find its deadline.
Action: get_activity_title: 5
PAUSE
Observation: Prática: Construindo um fluxo n8n(II)
Thought: Now that I have the title of activity 5, which is "Prática: Construindo um fluxo n8n(II)", I can use this information to find its deadline.
Action: get_activity_deadline: Prática: Construindo um fluxo n8n(II)
PAUSE
Observation: 3 dias
Thought: I have now found the deadline for activity 5, which is "3 dias".
Action: None needed, I have the answer.
Answer: The deadline for activity 5 is 3 days.


In [10]:
print("TESTE 2: Consultando múltiplas atividades")  
print("-" * 40)
# Pergunta complexa sobre múltiplas atividades
loop(query="What are the deadlines for activity 1 and activity 10? What is the total time for both?")

TESTE 2: Consultando múltiplas atividades
----------------------------------------
Thought: To find the total time for both activities, I need to first get the title of activity 1, then find its deadline, and do the same for activity 10. After that, I can add both deadlines together to get the total time.

Action: get_activity_title: 1
PAUSE
Observation: Vídeo: Fundamentos de Agentes de IA (I)
Thought: Now that I have the title of activity 1, I need to get its deadline. Then, I will repeat the process for activity 10.

Action: get_activity_deadline: Vídeo: Fundamentos de Agentes de IA (I)
PAUSEWidthSpaceSplitOptionsI will continue after this pause to get the deadline for activity 10 and calculate the total time.
Observation: 2 dias
Thought: Now that I have the deadline for activity 1, I need to get the title and deadline of activity 10. After that, I can add both deadlines together to get the total time.

Action: get_activity_title: 10
PAUSE
Observation: Pratica: Multi Agentes com ADK 

PRINCIPAIS CONCEITOS QUE APRENDI:

1. REACT = Reasoning (Thought) + Acting (Action)
   - LLM não só gera texto, mas pode "agir" no mundo
   - Cada ação é seguida de uma observação
   - Permite resolver problemas complexos passo a passo

2. SYSTEM PROMPT é fundamental
   - Define o "comportamento" do agente
   - Exemplos são cruciais para ensinar o padrão
   - Deve ser claro e detalhado

3. TOOLS expandem capacidades
   - Funções Python que o agente pode "chamar"
   - Podem acessar APIs, bancos de dados, etc.
   - Tornam o LLM mais poderoso que apenas geração de texto

4. LOOP é onde a mágica acontece
   - Automatiza a execução das actions
   - Simula um humano "assistindo" o agente trabalhar
   - Permite iteração até encontrar a resposta

5. HISTÓRICO DE CONVERSA
   - Mantém contexto durante toda a interação
   - Permite que o agente "lembre" de conversas anteriores
   - Essencial para conversas coerentes